In [1]:
%pwd

'e:\\Med GenAI\\MediAssist-AI-End-to-End-Medical-Chatbot\\research'

In [2]:
cd ..

e:\Med GenAI\MediAssist-AI-End-to-End-Medical-Chatbot


In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os
load_dotenv()

True

In [4]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GOOGLE_API_KEY=os.environ.get('GOOGLE_API_KEY')

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [5]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)

In [32]:
#Extract Data From the PDF File
from langchain_community.document_loaders import TextLoader
def load_pdf_file(data):
    loader= TextLoader(data)

    documents=loader.lazy_load()

    return documents

In [33]:
extracted_data=load_pdf_file(data="Data/Medical-book.pdf")

In [35]:
# len(extracted_data)

In [9]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [10]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [11]:
#Split the Data into Text Chunks
def text_split(minimal_docs):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    text_chunks=text_splitter.split_documents(minimal_docs)
    return text_chunks

In [12]:
text_chunks=text_split(minimal_docs)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5961


In [13]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
)

In [14]:
result = embeddings.embed_query("The text_field parameter sets the name of the metadata field that stores the raw text when you upsert records using a LangChain operation such as vectorstore.from_documents or vectorstore.add_texts. This metadata field is used as the page_content in the Document objects retrieved from query-like LangChain operations such as vectorstore.similarity_search. If you do not specify a value for text_field, it will default to")
len(result)

768

In [16]:
from pinecone import ServerlessSpec

index_name = "medical-bot"  # change if desired

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

In [17]:
# Embed each chunk and upsert the embeddings into your Pinecone index.

from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [37]:
for batch in extracted_data.lazy_load(batch_size=100):  
    index.upsert(batch) 

AttributeError: 'generator' object has no attribute 'lazy_load'

In [19]:
docsearch.add_documents(
    documents=text_chunks,
    batch_size=50  # Set your desired batch size here
)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Fri, 18 Jul 2025 18:43:02 GMT', 'Content-Type': 'application/json', 'Content-Length': '118', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '17418', 'x-pinecone-request-id': '357977155169826125', 'x-envoy-upstream-service-time': '2', 'server': 'envoy'})
HTTP response body: {"code":11,"message":"Error, message length too large: found 4386346 bytes, the limit is: 4194304 bytes","details":[]}
